In [56]:
# global var set
import transformers

# model info, change as needed
batch_size = 16
num_epochs = 16

model_checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
# fileTag = "original-plutchik-noCombin-v1"   # original - no Combine    - pure raw
fileTag = "original-plutchik-v1"             # original - w/ Combine
# fileTag = "clean-noCombin-v1"                # clean    - no Combine
# fileTag = "clean-v1"                         # clean    - w/ Combine    - pure clean

# Convert dataset to suitable format
IMPORTANT: please never run this section again if you have your dataset ready!

In [57]:
import pandas as pd
from pandas import DataFrame

trainDatasetOriginal = pd.read_csv(f'../../data/csv_version/dev/emotion/allcharlinepairs-{fileTag}.csv')
testDatasetOriginal = pd.read_csv(f'../../data/csv_version/test/emotion/allcharlinepairs-{fileTag}.csv')

In [58]:
trainDatasetProcessed = DataFrame({'emotion' : trainDatasetOriginal['emotion'],
                                   'plutchik' : trainDatasetOriginal['plutchik'],
                                  'selection0': pd.concat([trainDatasetOriginal['sentence'][:trainDatasetOriginal.shape[0]//2], trainDatasetOriginal.sample(frac = 1).reset_index()['sentence'][trainDatasetOriginal.shape[0]//2:]]), 
                                  'selection1': pd.concat([trainDatasetOriginal.sample(frac = 1).reset_index()['sentence'][:trainDatasetOriginal.shape[0]//2], trainDatasetOriginal['sentence'][trainDatasetOriginal.shape[0]//2:]]), 
                                  'label': pd.Series(0 if x < trainDatasetOriginal.shape[0]//2 else 1 for x in trainDatasetOriginal.index)}).sample(frac=1).reset_index(drop=True)

testDatasetProcessed = DataFrame({'emotion' : testDatasetOriginal['emotion'], 
                                  'plutchik' : testDatasetOriginal['plutchik'],
                                  'selection0': pd.concat([testDatasetOriginal['sentence'][:testDatasetOriginal.shape[0]//2], testDatasetOriginal.sample(frac = 1).reset_index()['sentence'][testDatasetOriginal.shape[0]//2:]]), 
                                  'selection1': pd.concat([testDatasetOriginal.sample(frac = 1).reset_index()['sentence'][:testDatasetOriginal.shape[0]//2], testDatasetOriginal['sentence'][testDatasetOriginal.shape[0]//2:]]), 
                                  'label': pd.Series(0 if x < testDatasetOriginal.shape[0]//2 else 1 for x in testDatasetOriginal.index)}).sample(frac=1).reset_index(drop=True)

In [59]:
trainDatasetProcessed.to_csv(f'./dataset/2Select-{fileTag}-train.csv')
trainDatasetProcessed

,emotion,plutchik,selection0,selection1,label
0,"['none', 'delighted', 'eager', 'happy']","{'joy': 2, 'trust': 2, 'fear': 0, 'surprise': ...",Daniel began to wonder if anyone had ever coll...,Anna ordered extra spicy chicken wings from he...,1
1,"['admired', 'happy', 'joy']","{'joy': 3, 'trust': 1, 'fear': 0, 'surprise': ...",Lauren's mother even decorated the cake with c...,Sam is happy!,0
2,"['none', 'disgusted', 'disgust', 'anger', 'mad']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",She quit listening to the radio show.,Jane wondered if she had made a mistake moving...,0
3,"['none', 'inspired', 'intrigued']","{'joy': 1, 'trust': 0, 'fear': 0, 'surprise': ...",She'd just built her new house and it was her ...,John watched a Food Network show on making gua...,1
4,"['none', 'excluded', 'disappointed', 'sad', 's...","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...","Sadly, she cannot drink Diet Coke while pregnant.",No sane dog chases a pack of wild coyotes.,0
...,...,...,...,...,...
12410,"['Taken advantage of', 'comfortable', 'none']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",James figured it was worth it.,He started to tell the joke and realized he fo...,0
12411,"['happy', 'resourceful', 'accomplished', 'exci...","{'joy': 2, 'trust': 1, 'fear': 0, 'surprise': ...",He mailed it to North Pole.,"Bob decided to start a bar, and hired Jerry to...",1
12412,"['none', 'proud', 'athletic', 'entertained', '...","{'joy': 1, 'trust': 0, 'fear': 0, 'surprise': ...",Her brothers all played baseball on little lea...,Jerry came home with two because he couldn't d...,0
12413,"['obligated', 'determined', 'doubtful', 'anxio...","{'joy': 1, 'trust': 1, 'fear': 1, 'surprise': ...",Her mother brought her to the eye doctor.,"Meagan didn't think she could do it right, but...",1


In [60]:
testDatasetProcessed.to_csv(f'./dataset/2Select-{fileTag}-test.csv')
testDatasetProcessed

,emotion,plutchik,selection0,selection1,label
0,"['concerned', 'sympathetic', 'sad']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",It tasted terrible.,Finally she saw there was a car accident.,1
1,"['unconcerned', 'forgetful', 'angry', 'annoyed']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",The boy saw a crab.,Sometimes her brother forgets to knock before ...,1
2,"['fatigued', 'fed up', 'tired']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",He realized how tired he was and turned around...,Alvin was always late with the rent.,0
3,"['sad', 'lonely', 'disappointed', 'alone', 'Sa...","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",They liked basketball or football.,She was so excited about getting a phone at last.,0
4,"['worried', 'clumsy', 'none']","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",I accidentally bumped it.,"Within hours, Marla was up and walking around.",0
...,...,...,...,...,...
11845,"['rushed', 'anxious']","{'joy': 0, 'trust': 0, 'fear': 1, 'surprise': ...",Sabrina was in a hurry.,It made my wrists hurt badly.,0
11846,"['confident', 'involved', 'competitive']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ...",Tanice had an inflated sense of self worth.,The pitcher threw the ball towards him.,1
11847,"['surprised', 'unaware', 'Guilt', 'embarassed'...","{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': ...",He thought about which activities brought him ...,Just as she is buying the soda a friend almost...,1
11848,"['loved', 'sad', 'honored', 'relief']","{'joy': 1, 'trust': 1, 'fear': 0, 'surprise': ...",She kept Maggie until she passed away.,Lucy learned to try harder when cleaning next ...,0


# load dataset

In [40]:
from datasets import load_dataset, load_metric

In [41]:
dataset = load_dataset('csv', data_files={'train': f'./dataset/2Select-{fileTag}-train.csv', 
                                           'test': f'./dataset/2Select-{fileTag}-test.csv'})

Using custom data configuration default-66134232cfd9d874


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\JAM_0\.cache\huggingface\datasets\csv\default-66134232cfd9d874\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'emotion', 'plutchik', 'selection0', 'selection1', 'label'],
        num_rows: 42954
    })
    test: Dataset({
        features: ['Unnamed: 0', 'emotion', 'plutchik', 'selection0', 'selection1', 'label'],
        num_rows: 43195
    })
})

In [43]:
dataset['test'][0]

{'Unnamed: 0': 0,
 'emotion': "['disappointed']",
 'plutchik': "{'joy': 0, 'trust': 0, 'fear': 0, 'surprise': 0, 'sadness': 3, 'disgust': 2, 'anger': 2, 'anticipation': 0}",
 'selection0': 'I heard a crack but when I checked the phone it still worked perfect.',
 'selection1': 'I lost the game very badly.',
 'label': 1}

In [44]:
def show_one(example):
    print(f"Context: {example['plutchik']}")
    print(f"  A - {example['selection0']}")
    print(f"  B - {example['selection1']}")
    print(f"\nGround truth: option {['A', 'B'][example['label']]}")

In [45]:
show_one(dataset["train"][0])

Context: {'joy': 2, 'trust': 0, 'fear': 0, 'surprise': 2, 'sadness': 0, 'disgust': 0, 'anger': 0, 'anticipation': 2}
  A - I woke up one morning and went outside to grab the newspaper.
  B - It was beautiful under the water.

Ground truth: option B


# Preprocessing Data

In [46]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
from transformers import DistilBertTokenizer, DistilBertForMultipleChoice
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")

In [47]:
import re
selectionList = ["selection0", "selection1"]
weightRemap = ["NOT ", "LITTLE ", "", "VERY ", "ABSOLUTELY "]
def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    # first_sentences = [["The following sentences contain emotions: {}".format(context.strip("[").strip("]").replace('\'', '')) ]*2 for context in examples["emotion"] ]
    # first_sentences = [["The following sentences contain emotions: {}".format(context.strip("[").strip("]").replace('\"', '')) ]*2 for context in examples["plutchik"] ]
    first_sentences = [["The following sentences contain emotions: {}".format(", ".join([weightRemap[int(eachCaseWeight.replace("]", "").replace("[", "").replace("}", "").replace("{", "").replace("\"", "").replace("\'", ""))] 
                                                                      + eachCaseEmotionType.replace("]", "").replace("[", "").replace("}", "").replace("{", "").replace("\"", "").replace("\'", "").strip()
                        for eachCaseWeight, eachCaseEmotionType in 
                        zip([re.split(':|,',eachEmotionCombination)[1::2] for eachEmotionCombination in examples["plutchik"]][eventIndex], 
                           [re.split(':|,',eachEmotionCombination)[::2] for eachEmotionCombination in examples["plutchik"]][eventIndex])]))]*2 for eventIndex in 
                           range(len([re.split(':|,',eachEmotionCombination)[1::2] for eachEmotionCombination in examples["plutchik"]]))]
    
    # first_sentences = [["The following sentences contain emotions: {}".format(', '.join([(weightRemap[eachEmotion[1]] + " " +eachEmotion[0]).strip() 
    #                    for eachEmotion in ast.literal_eval(context).items()]))]*2 
    #                    for context in examples["plutchik"]]
    # Grab all second sentences possible for each context.
    second_sentences = [[examples[selection][index] for selection in selectionList]for index in range(len(examples['selection0']))]

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    # print(tokenized_examples.items())
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [48]:
examples = dataset["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 2 [47, 41]


In [49]:
[tokenizer.decode(features["input_ids"][a][i]) for a in range(5) for i in range(1) ]

['[CLS] The following sentences contain emotions : joy, NOT trust, NOT fear, surprise, NOT sadness, NOT disgust, NOT anger, anticipation [SEP] I woke up one morning and went outside to grab the newspaper. [SEP]',
 '[CLS] The following sentences contain emotions : VERY joy, trust, NOT fear, surprise, NOT sadness, NOT disgust, NOT anger, NOT anticipation [SEP] At the end of the party, the kids went home for the day. [SEP]',
 '[CLS] The following sentences contain emotions : joy, trust, NOT fear, surprise, NOT sadness, NOT disgust, NOT anger, VERY anticipation [SEP] He then saw his parents putting gifts under the tree. [SEP]',
 '[CLS] The following sentences contain emotions : NOT joy, NOT trust, NOT fear, NOT surprise, NOT sadness, NOT disgust, NOT anger, NOT anticipation [SEP] After all of my hard work I became a professional baseball player! [SEP]',
 '[CLS] The following sentences contain emotions : NOT joy, NOT trust, VERY fear, VERY surprise, VERY sadness, VERY disgust, VERY anger, N

In [50]:
encoded_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/43 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [ ]:
# from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
# model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForMultipleChoice
import torch
model = DistilBertForMultipleChoice.from_pretrained("distilbert-base-cased")

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-emotionCommonsense",
    evaluation_strategy = "epoch",
    learning_rate=5e-7, # for bert-base
    # learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

In [ ]:
show_one(dataset["train"][8])

# Trainer Defined

In [ ]:
import numpy as np

valStored = []
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    valStored.append((preds != label_ids).astype(np.float32));
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

出现validation loss 上升情况大多是训练集验证集数据分布不一致，或者训练集过小，未包含验证集中所有情况，
也就是过拟合导致的。而解决这种现象可以尝试以下几种策略：
1. 增加训练样本增加正则项系数权重，
2. 减小过拟合加入早停机制，ValLoss上升几个epoch直接停止
3. 采用Focal Loss
4. 加入Label Smoothing

# Store Value

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

Z = np.transpose(valStored)

fig, ax = plt.subplots()
ax.pcolormesh(Z)

In [82]:
import pandas as pd

dataLog = pd.DataFrame(trainer.state.log_history)
dataLog.to_csv(f'./trainingMetric/[Plutchik] 2Select/TI-{model_checkpoint}-{fileTag}.csv')

In [83]:
evaluationIterationResult = pd.DataFrame(np.transpose(valStored))
evaluationIterationResult.to_csv(f'./trainingMetric/[Plutchik] 2Select/ESI-{model_checkpoint}-{fileTag}.csv')